In [9]:
!pip install dgl
import dgl
from dgl.nn.pytorch import GATConv

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)
        self.reset_parameters()

    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.fc.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)

    def edge_attention(self, edges):
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        return {'e': F.leaky_relu(a)}

    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):

        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}

    def forward(self, h):
        z = self.fc(h)
        self.g.ndata['z'] = z
        self.g.apply_edges(self.edge_attention)
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')

In [0]:
def edge_attention(self, edges):
    z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
    a = self.attn_fc(z2)
    return {'e' : F.leaky_relu(a)}

In [0]:
def reduce_func(self, nodes):
 
    alpha = F.softmax(nodes.mailbox['e'], dim=1)
    h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
    return {'h' : h}

In [0]:
class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, num_heads, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
        self.merge = merge

    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            return torch.cat(head_outs, dim=1)
        else:
            return torch.mean(torch.stack(head_outs))

In [0]:
class GAT(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1 = MultiHeadGATLayer(g, in_dim, hidden_dim, num_heads)
        self.layer2 = MultiHeadGATLayer(g, hidden_dim * num_heads, out_dim, 1)

    def forward(self, h):
        h = self.layer1(h)
        h = F.elu(h)
        h = self.layer2(h)
        return h

In [0]:
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh
import networkx as nx

def load_cora_data():
    data = citegrh.load_cora()
    features = torch.FloatTensor(data.features)
    labels = torch.LongTensor(data.labels)
    mask = torch.BoolTensor(data.train_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, mask

In [16]:
import time
import numpy as np

g, features, labels, mask = load_cora_data()

net = GAT(g,
          in_dim=features.size()[1],
          hidden_dim=8,
          out_dim=7,
          num_heads=2)

optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

dur = []
for epoch in range(30):
    if epoch >= 3:
        t0 = time.time()

    logits = net(features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[mask], labels[mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
        epoch, loss.item(), np.mean(dur)))

Extracting file to /root/.dgl/cora


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 1.9473 | Time(s) nan
Epoch 00001 | Loss 1.9451 | Time(s) nan
Epoch 00002 | Loss 1.9430 | Time(s) nan
Epoch 00003 | Loss 1.9409 | Time(s) 0.1095
Epoch 00004 | Loss 1.9388 | Time(s) 0.1053
Epoch 00005 | Loss 1.9367 | Time(s) 0.1029
Epoch 00006 | Loss 1.9345 | Time(s) 0.1018
Epoch 00007 | Loss 1.9324 | Time(s) 0.1020
Epoch 00008 | Loss 1.9303 | Time(s) 0.1022
Epoch 00009 | Loss 1.9282 | Time(s) 0.1022
Epoch 00010 | Loss 1.9260 | Time(s) 0.1018
Epoch 00011 | Loss 1.9239 | Time(s) 0.1013
Epoch 00012 | Loss 1.9217 | Time(s) 0.1014
Epoch 00013 | Loss 1.9196 | Time(s) 0.1027
Epoch 00014 | Loss 1.9174 | Time(s) 0.1024
Epoch 00015 | Loss 1.9153 | Time(s) 0.1022
Epoch 00016 | Loss 1.9131 | Time(s) 0.1018
Epoch 00017 | Loss 1.9109 | Time(s) 0.1018
Epoch 00018 | Loss 1.9087 | Time(s) 0.1017
Epoch 00019 | Loss 1.9065 | Time(s) 0.1014
Epoch 00020 | Loss 1.9043 | Time(s) 0.1014
Epoch 00021 | Loss 1.9021 | Time(s) 0.1011
Epoch 00022 | Loss 1.8998 | Time(s) 0.1010
Epoch 00023 | Loss 1